googleドライブ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


huggingface

In [ ]:
!pip install huggingface_hub

from google.colab import userdata
from huggingface_hub import login

# Secretsからトークンを取得
HF_TOKEN = userdata.get('HF_TOKEN')
login(HF_TOKEN)

wandb

In [2]:
from google.colab import userdata
import wandb

# Colabシークレットから取得
wandb_api_key = userdata.get('WANDB_API_KEY')
wandb.login(key=wandb_api_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: puwaer (puwaer-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

ライブラリ

In [ ]:
!pip uninstall unsloth -y
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --upgrade torch
!pip install --upgrade xformers

import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-j64zzec_/unsloth_2b9d153bd434437ba766faa2ef95aa2c
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-j64zzec_/unsloth_2b9d153bd434437ba766faa2ef95aa2c
  Resolved https://github.com/unslothai/unsloth.git to commit e80d642bc777f7a219bdd34aea1a77751f066785
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 305.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 352.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 318.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.9/126.9 kB 333.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 267.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.7/123.7 kB 331.4 MB/s eta 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 15.0 MB/s eta 0:00:00


パラメータ

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset


max_seq_length = 2048                                    # コンテキスト長
dtype = None                                            # Noneにしておけば自動で設定
load_in_4bit = True                                     # 量子化 True

#model_id = "llm-jp/llm-jp-3-3.7b"
model_id = "/content/drive/MyDrive/program/code_translation/base_model/llm-jp-3-1.8b"
model_adapter = "test_2"
dataset = load_dataset("json", data_files="/content/drive/MyDrive/program/code_translation/data/ichikara-instruction-003-001-test.json")


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    max_seq_length = max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    #trust_remote_code=True,
)


model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


# 学習時のプロンプトフォーマットの定義
prompt = """### 指示
{}
### 回答
{}"""


EOS_TOKEN = tokenizer.eos_token # トークナイザーのEOSトークン（文末トークン）
def formatting_prompts_func(examples):
    input = examples["text"] # 入力データ
    output = examples["output"] # 出力データ
    text = prompt.format(input, output) + EOS_TOKEN # プロンプトの作成
    return { "formatted_text" : text, } # 新しいフィールド "formatted_text" を返す
pass

# # 各データにフォーマットを適用
dataset = dataset.map(
    formatting_prompts_func,
    num_proc= 4,                      # 並列処理数を指定
)

dataset

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.18 patched 24 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Map (num_proc=4):   0%|          | 0/50 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['ID', 'text', 'output', 'formatted_text'],
        num_rows: 50
    })
})

In [ ]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template


# トークナイザーにLLaMA-3.1のチャットテンプレートを適用
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

# 元のプロンプトフォーマット
prompt = """### 指示
{}
### 回答
{}"""

EOS_TOKEN = tokenizer.eos_token  # トークナイザーのEOSトークン（文末トークン）

def formatting_prompts_func(examples):
    inputs = examples["text"]    # 入力データ
    outputs = examples["output"] # 出力データ
    
    # 会話形式に変換（LLaMAのチャットテンプレート用）
    convos = []
    for input_text, output_text in zip(inputs, outputs):
        convo = [
            {"role": "user", "content": input_text},
            {"role": "assistant", "content": output_text}
        ]
        convos.append(convo)
    
    # チャットテンプレートを適用
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) + EOS_TOKEN 
            for convo in convos]
    
    return {"formatted_text": texts}

# 各データにフォーマットを適用
dataset = dataset.map(
    formatting_prompts_func,
    num_proc=4,  # 並列処理数を指定
)

print(dataset)

In [ ]:
"""
training_arguments: 学習の設定

  - output_dir:
      -トレーニング後のモデルを保存するディレクトリ

  - per_device_train_batch_size:
      - デバイスごとのトレーニングバッチサイズ

  - per_device_eval_batch_size:
      - デバイスごとの評価バッチサイズ

  - gradient_accumulation_steps:
      - 勾配を更新する前にステップを積み重ねる回数

  - optim:
      - オプティマイザの設定

  - num_train_epochs:
      - エポック数

  - eval_strategy:
      - 評価の戦略 ("no"/"steps"/"epoch")

  - eval_steps:
      - eval_strategyが"steps"のとき、評価を行うstep間隔

  - logging_strategy:
      - ログ記録の戦略

  - logging_steps:
      - ログを出力するステップ間隔

  - warmup_steps:
      - 学習率のウォームアップステップ数

  - save_steps:
      - モデルを保存するステップ間隔

  - save_total_limit:
      - 保存しておくcheckpointの数

  - max_steps:
      - トレーニングの最大ステップ数

  - learning_rate:
      - 学習率

  - fp16:
      - 16bit浮動小数点の使用設定

  - bf16:
      - BFloat16の使用設定

  - group_by_length:
      -  入力シーケンスの長さによりバッチをグループ化 (トレーニングの効率化)

  - report_to:
      - ログの送信先 ("wandb"/"tensorboard"など)
"""
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = model_adapter,             #wandbのプロジェクトの名前
        report_to = "wandb",
    ),
)


from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)


Unsloth: Tokenizing ["formatted_text"] (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

学習

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 1 | Total steps = 6
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 33,816,576/8,000,000,000 (0.42% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss


保存

In [ ]:
# huggingfaceにLoRAアダプタだけ保存
model.push_to_hub_merged(
    model_adapter,
    tokenizer=tokenizer,
    save_method="lora",
    token=HF_TOKEN,
    private=True
)

In [ ]:
# google driveにLoRAアダプタだけ保存
model.save_pretrained(
    f"/content/drive/MyDrive/program/code_translation/model_adapterl/{model_adapter}",
    save_method="lora"
)
tokenizer.save_pretrained(f"/content/drive/MyDrive/program/code_translation/model_adapter/{model_adapter}")


('/content/drive/MyDrive/program/code_translation/model/test_2/tokenizer_config.json',
 '/content/drive/MyDrive/program/code_translation/model/test_2/special_tokens_map.json',
 '/content/drive/MyDrive/program/code_translation/model/test_2/tokenizer.json')